<a href="https://colab.research.google.com/github/diana-07/Diana_tests_DL/blob/main/prepare_data_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y mediapipe-silicon
!pip uninstall -y mediapipe
!pip uninstall -y protobuf
!pip install protobuf==3.20.3  # Specific version that often works well with MediaPipe
!pip install mediapipe


Found existing installation: protobuf 5.29.4
Uninstalling protobuf-5.29.4:
  Successfully uninstalled protobuf-5.29.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 29.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 4.25.7 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
grpcio

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score
import shutil
import random
import mediapipe as mp

from collections import defaultdict
from sklearn.model_selection import train_test_split

In [ ]:
def convert_video_to_npy(video_path, resize_shape=(224, 224)):
    """
    Converts a video to a numpy array of resized frames.

    Parameters:
    - video_path: The path to the video file.
    - resize_shape: The desired frame size (default is 224x224).

    Returns:
    - frames_array: Numpy array containing all frames.
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    if not cap.isOpened():
        raise ValueError(f"Error opening video file: {video_path}")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to the desired shape
        frame_resized = cv2.resize(frame, resize_shape)
        frames.append(frame_resized)

    cap.release()

    # Convert list of frames into a numpy array
    frames_array = np.array(frames)
    return frames_array

def create_subfolders(pastas_dir, output_dir):
    """
    Creates subfolders in the output directory and processes .avi files.

    Parameters:
    - pastas_dir: The root directory containing class folders.
    - output_dir: The directory where the .npy files will be saved.
    """
    for pastas_filename in os.listdir(pastas_dir):
        origem_path = os.path.join(pastas_dir, pastas_filename)

        # Check if it's a directory (ignore files)
        if os.path.isdir(origem_path):
            nova_pasta_path = os.path.join(output_dir, pastas_filename)
            os.makedirs(nova_pasta_path, exist_ok=True)

            # Process each video file in the subfolder
            videos_dir = os.path.join(pastas_dir, pastas_filename)
            for video_filename in os.listdir(videos_dir):
                if video_filename.endswith(".avi"):  # Process only .avi files
                    video_path = os.path.join(videos_dir, video_filename)
                    video_name = os.path.splitext(video_filename)[0]

                    try:
                        # Convert the video to numpy array
                        frames_array = convert_video_to_npy(video_path)

                        # Save the numpy array to a .npy file
                        output_filename = os.path.join(nova_pasta_path, f"{video_name}.npy")
                        np.save(output_filename, frames_array)
                    except ValueError as e:
                        print(f"Error processing {video_filename}: {e}")
                    except Exception as e:
                        print(f"Unexpected error with {video_filename}: {e}")

In [ ]:
# Pad/truncate to 120 frames
def pad_or_truncate_keypoints(keypoints, target_length=120):
    """
    Pads or truncates a keypoints array to a fixed number of frames (target_length).
    If padding is needed, uses zeros.
    """
    current_length = keypoints.shape[0]

    if current_length == target_length:
        return keypoints
    elif current_length > target_length:
        return keypoints[:target_length]  # Truncate
    else:
        # Pad with zeros
        padding = np.zeros((target_length - current_length, keypoints.shape[1], keypoints.shape[2]))
        return np.concatenate([keypoints, padding], axis=0)

In [ ]:
def extract_player_id(filename):
    # Example: 'player1_video1.npy' → 'player1'
    return filename.split('_')[0]

def collect_by_players(player_ids):
    dataset = []
    for pid in player_ids:
        for action, filename in player_video_map[pid]:
            keypoints = all_keypoints[action][filename]
            dataset.append((keypoints, action))
    return dataset